In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Dataset/'Customer Escalations' 


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Dataset/Customer Escalations


In [2]:
import pandas as pd
import numpy as np
import random
from collections import Counter
import gensim
from gensim.models import Doc2Vec
import seaborn as sns

In [3]:
challenge=pd.read_csv('challenge_dictionary_info.csv')
case_status=pd.read_csv('IBI_case_status_history_v2.csv')
case_comments=pd.read_csv('IBI_case_comments_anonymized.csv')
case_metadata=pd.read_csv('IBI_case_metadata_anonymized.csv')
case_milestones=pd.read_csv('IBI_case_milestones_anonymized.csv')
id2_lemma=pd.read_csv("id_to_lemma_public_translations.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
   

In [5]:
corpus=[]
tmp_dict=dict(zip(challenge['id'].values,challenge['pos'].values))
tmp_dict1=dict(zip(id2_lemma['id'].values,id2_lemma['lemma'].values))
pos=[]
gram=['ADP','PROPN','PUNCT','PRON','DET','CCONJ','AUX','SCONJ','SPACE','ADV']

for cols in ['NOTEDESCRIPTION','MILESTONEDESCRIPTION']:
  tmp=case_milestones[cols].values;pos=[]
  for val in tmp:
    tmp1=[k for k in val.split(" ")  if tmp_dict[k] not in gram and k not in id2_lemma['id'].values]
    pos.append(tmp1)
  corpus.extend(pos)
tmp=case_comments['NOTES'].values;pos=[]
for val in tmp:
    tmp1=[k for k in val.split(" ")  if tmp_dict[k] not in gram and k not in id2_lemma['id'].values]
    pos.append(tmp1)
corpus.extend(pos)

len(corpus)

2759523

In [7]:
%%time

embed_size=100

def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

train_txt = list(create_tagged_document(corpus))

print(train_txt[:1])
model = Doc2Vec(train_txt,vector_size=embed_size, min_count=1, epochs=30,workers=1)#,compute_loss=True)
model.build_vocab(train_txt,update=True)
model.train(train_txt, total_examples=model.corpus_count, epochs=model.epochs)


[TaggedDocument(words=['ZLkp', 'xdE5', 'GRZz', 'wRvE', 'fZhn', 'nrKA', '2nEL', 'cutI', '4cBi', 'RqRA', 'tcau', 'jWkl', 'x6v9', 'Me8P', 'cyvr', 'UGdF', 'ZLkp', 'xdE5', 'GRZz', '24Nu', 'kAx1', 'VrUP', 'F6CA', 'yZUr', '24Nu', 'jvez', 'ycvM', 'F6CA', 'z2i6', 'pl5b', 'vQUg', 'r0eT'], tags=[0])]
CPU times: user 15.6 s, sys: 404 ms, total: 16 s
Wall time: 16 s


In [ ]:
from google.colab import files
model.save('Doc2vec_trained')
files.download('Doc2vec_trained')



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tmp=case_milestones['NOTEDESCRIPTION'].values;pos=[]
tmp_dict=dict(zip(challenge['id'].values,challenge['pos'].values))
pos=[]
gram=['ADP','PROPN','PUNCT','PRON','DET','CCONJ','AUX','SCONJ','SPACE','ADV']

for val in tmp:
    tmp1=[k for k in val.split(" ")  if tmp_dict[k] not in gram and k not in id2_lemma['id'].values]
    pos.append(model.infer_vector(tmp1))
  
df=pd.DataFrame(pos,columns=[str(val)+'_notesdesc' for val in range(0,embed_size)])
df = reduce_mem_usage(df)
df.to_csv('notesdesc_embed.csv')
df


In [ ]:
tmp=case_milestones['MILESTONESDESCRIPTION'].values;pos=[]
tmp_dict=dict(zip(challenge['id'].values,challenge['pos'].values))
pos=[]
gram=['ADP','PROPN','PUNCT','PRON','DET','CCONJ','AUX','SCONJ','SPACE','ADV']

for val in tmp:
    tmp1=[k for k in val.split(" ")  if tmp_dict[k] not in gram and k not in id2_lemma['id'].values]
    pos.append(model.infer_vector(tmp1))
  
df=pd.DataFrame(pos,columns=[str(val)+'_milesdesc' for val in range(0,embed_size)])
df = reduce_mem_usage(df)
df.to_csv('notesdesc_embed.csv')



In [ ]:
tmp=case_comments['NOTES'].values;pos=[]
tmp_dict=dict(zip(challenge['id'].values,challenge['pos'].values))
pos=[]
gram=['ADP','PROPN','PUNCT','PRON','DET','CCONJ','AUX','SCONJ','SPACE','ADV']

for val in tmp:
    tmp1=[k for k in val.split(" ")  if tmp_dict[k] not in gram and k not in id2_lemma['id'].values]
    pos.append(model.infer_vector(tmp1))
  


df=pd.DataFrame(pos,columns=[str(val)+'_comment' for val in range(0,embed_size)])
df = reduce_mem_usage(df)
df.to_csv('comment_embed.csv')
